In [1]:
import os
import numpy as np
import pandas as pd

from pandas import ExcelWriter
from pandas import ExcelFile

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from matplotlib.dates import DateFormatter
from matplotlib.dates import HourLocator
import matplotlib.dates as mdates

from scipy.interpolate import interp1d
import scipy.interpolate as spi
from scipy.interpolate import splrep, splev

import statistics as s

In [2]:
file_dir = os.path.join('/Users/digisha/Downloads/Fatigue_project/data')
file_name = os.path.join(file_dir, 'Sample_Data.xlsx')
df = pd.read_excel(file_name, sheetname='Sheet1')
df.head()

/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


,subject_id,question_id,answer,timestamp
0,6,1600,5,06-22-2018 22:18:46
1,6,1601,3,06-22-2018 22:18:55
2,6,1602,3,06-22-2018 22:19:02
3,6,1603,4,06-22-2018 22:19:26
4,6,1600,6,06-22-2018 23:44:43


In [3]:
#changing datatypes, time stamp in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [4]:
#only depression fatigue questionnare considererd
mask = (df['question_id'] >= 1600) & (df['question_id'] < 1700)
df_16 = df.loc[mask].reset_index(drop=True)
df_16['time'] = [t.time() for t in df_16['timestamp']]
#df_16['time'] = df_16['time'].apply(lambda x: x.strftime('%H:%M:%S'))
df_16['answer']=df_16.answer.astype('int64')
df_16.head()

,subject_id,question_id,answer,timestamp,time
0,6,1600,5,2018-06-22 22:18:46,22:18:46
1,6,1601,3,2018-06-22 22:18:55,22:18:55
2,6,1602,3,2018-06-22 22:19:02,22:19:02
3,6,1603,4,2018-06-22 22:19:26,22:19:26
4,6,1600,6,2018-06-22 23:44:43,23:44:43


In [5]:
df_16['time'] = df_16['time'].apply(lambda x: x.strftime('%H:%M:%S'))

In [6]:
df_16['question_id'].corr(df_16['answer'])

-0.396150738624947

In [7]:
#ndf = df_16.loc[df_16['subject_id'] == 6].reset_index(drop=True)

In [8]:
# date column added to know if the questionnaire is a part of the same day or the next
df_16['date'] = [d.date() for d in df_16['timestamp']]

In [9]:
# new columns added to convert timestamp to reading number
df_16['day'] = 0
df_16['reading'] = 0
df_16.head()

,subject_id,question_id,answer,timestamp,time,date,day,reading
0,6,1600,5,2018-06-22 22:18:46,22:18:46,2018-06-22,0,0
1,6,1601,3,2018-06-22 22:18:55,22:18:55,2018-06-22,0,0
2,6,1602,3,2018-06-22 22:19:02,22:19:02,2018-06-22,0,0
3,6,1603,4,2018-06-22 22:19:26,22:19:26,2018-06-22,0,0
4,6,1600,6,2018-06-22 23:44:43,23:44:43,2018-06-22,0,0


In [10]:
#count = number of rows for each patient
#patient = list of subject_id (when continuous loop in a for loop with range of subject_id)
#reset day and reading when the sunject_id in the data changes
# days-  to check if the day has changes
# minutes -  if Min>30, the question is considered to be a part of the sencond survey/questionnarie 
        # - reading number is one plus the previous one
# Reading : reset (=0) when the day changes (considering a new survey or questionnire)
# Days > 0: day = previous day + 1, else day = previous day 

patient = [6,27]
for pat in patient:
    count = 0
    #print(pat)
    for i in range(0,len(df_16)):
        if pat == df_16['subject_id'][i]:
            if count == 0:
                df_16['day'][i] = 0
                df_16['reading'][i] = 0
            else:
                days = (df_16['date'][i] - df_16['date'][i - 1]).days
                minutes = (df_16['timestamp'][i] - df_16['timestamp'][i - 1]).seconds//60
                
                if minutes < 30:
                    df_16['reading'].loc[i] = df_16['reading'].loc[i - 1]
                    df_16['day'].loc[i] = df_16['day'].loc[i - 1]
                else:
                    if days > 0:
                        #print(i, k)
                        df_16['day'].loc[i] = df_16['day'].loc[i-1] + 1
                        df_16['reading'].loc[i] = 0
                    else:
                        df_16['day'].loc[i] = df_16['day'].loc[i - 1]
                        df_16['reading'].loc[i] = df_16['reading'].loc[i - 1] + 1
         
            count += 1
        else:
            continue
    #print(count)
#df.head(340)
df_16.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,subject_id,question_id,answer,timestamp,time,date,day,reading
0,6,1600,5,2018-06-22 22:18:46,22:18:46,2018-06-22,0,0
1,6,1601,3,2018-06-22 22:18:55,22:18:55,2018-06-22,0,0
2,6,1602,3,2018-06-22 22:19:02,22:19:02,2018-06-22,0,0
3,6,1603,4,2018-06-22 22:19:26,22:19:26,2018-06-22,0,0
4,6,1600,6,2018-06-22 23:44:43,23:44:43,2018-06-22,0,1


In [11]:
#and answer from string to integer
df_16['answer']=df_16.answer.astype('int64')

In [12]:
df_16.tail(10)

,subject_id,question_id,answer,timestamp,time,date,day,reading
594,27,1602,3,2018-11-20 18:48:59,18:48:59,2018-11-20,15,3
595,27,1603,0,2018-11-20 18:49:03,18:49:03,2018-11-20,15,3
596,27,1600,5,2018-11-20 19:57:36,19:57:36,2018-11-20,15,4
597,27,1601,4,2018-11-20 19:57:40,19:57:40,2018-11-20,15,4
598,27,1602,2,2018-11-20 19:57:43,19:57:43,2018-11-20,15,4
599,27,1603,0,2018-11-20 19:57:46,19:57:46,2018-11-20,15,4
600,27,1600,5,2018-11-21 05:25:07,05:25:07,2018-11-21,16,0
601,27,1601,3,2018-11-21 05:25:10,05:25:10,2018-11-21,16,0
602,27,1602,1,2018-11-21 05:25:14,05:25:14,2018-11-21,16,0
603,27,1603,0,2018-11-21 05:25:17,05:25:17,2018-11-21,16,0


In [13]:
#df_scores = df.groupby(['time_id', 'pat_id'])['ans'].sum().reset_index()
#t1 = pd.pivot_table(df, values='ans', index=['time_id'], columns=['pat_id'], aggfunc=np.sum, fill_value=0)

In [14]:
df_16['srno'] = df_16['day']*10 + df_16['reading']
df_16.head(10)

,subject_id,question_id,answer,timestamp,time,date,day,reading,srno
0,6,1600,5,2018-06-22 22:18:46,22:18:46,2018-06-22,0,0,0
1,6,1601,3,2018-06-22 22:18:55,22:18:55,2018-06-22,0,0,0
2,6,1602,3,2018-06-22 22:19:02,22:19:02,2018-06-22,0,0,0
3,6,1603,4,2018-06-22 22:19:26,22:19:26,2018-06-22,0,0,0
4,6,1600,6,2018-06-22 23:44:43,23:44:43,2018-06-22,0,1,1
5,6,1601,5,2018-06-22 23:44:59,23:44:59,2018-06-22,0,1,1
6,6,1602,3,2018-06-22 23:45:07,23:45:07,2018-06-22,0,1,1
7,6,1603,5,2018-06-22 23:45:20,23:45:20,2018-06-22,0,1,1
8,6,1600,0,2018-06-23 06:08:06,06:08:06,2018-06-23,1,0,10
9,6,1601,5,2018-06-23 06:08:20,06:08:20,2018-06-23,1,0,10


In [15]:
#mask = (df_16['subject_id'] == 6)
df_16_6 = df_16.loc[df_16['subject_id'] == 6].reset_index(drop=True)

In [16]:
df_16_6.tail()

,subject_id,question_id,answer,timestamp,time,date,day,reading,srno
331,6,1603,2,2018-07-07 15:54:07,15:54:07,2018-07-07,15,2,152
332,6,1600,6,2018-07-07 19:58:00,19:58:00,2018-07-07,15,3,153
333,6,1601,2,2018-07-07 19:58:04,19:58:04,2018-07-07,15,3,153
334,6,1602,6,2018-07-07 19:58:08,19:58:08,2018-07-07,15,3,153
335,6,1603,3,2018-07-07 19:58:13,19:58:13,2018-07-07,15,3,153


In [17]:
df1 = df_16_6.pivot_table(index='srno', columns='question_id', values='answer', aggfunc=np.mean)

In [18]:
df1.head()

question_id,1600,1601,1602,1603
srno,,,,
0,5,3,3,4
1,6,5,3,5
10,0,5,0,2
11,0,0,0,1
12,2,0,0,2


In [19]:
b = df1.corr()

#df1.corr(method='spearman')
#df1.corr(method='kendall')

In [20]:
df_16_27 = df_16.loc[df_16['subject_id'] == 27].reset_index(drop=True)
df2 = df_16_27.pivot_table(index='srno', columns='question_id', values='answer', aggfunc=np.mean)
a = df2.corr()

In [21]:
df_16_6.index

RangeIndex(start=0, stop=336, step=1)

In [22]:
#stats
df_16_6.head(7)

,subject_id,question_id,answer,timestamp,time,date,day,reading,srno
0,6,1600,5,2018-06-22 22:18:46,22:18:46,2018-06-22,0,0,0
1,6,1601,3,2018-06-22 22:18:55,22:18:55,2018-06-22,0,0,0
2,6,1602,3,2018-06-22 22:19:02,22:19:02,2018-06-22,0,0,0
3,6,1603,4,2018-06-22 22:19:26,22:19:26,2018-06-22,0,0,0
4,6,1600,6,2018-06-22 23:44:43,23:44:43,2018-06-22,0,1,1
5,6,1601,5,2018-06-22 23:44:59,23:44:59,2018-06-22,0,1,1
6,6,1602,3,2018-06-22 23:45:07,23:45:07,2018-06-22,0,1,1


In [23]:
count_1600 = 0
count_1601 = 0
count_1602 = 0
count_1603 = 0

for i in range(0,len(df_16_6)):
    ques = df_16_6["question_id"][i]
    if ques == 1600:
        count_1600 +=1
    elif ques == 1601:
        count_1601 += 1
    elif ques == 1602:
        count_1602 += 1
    elif ques == 1603:
        count_1603 += 1
    else:
        print('stray', i)
list = [count_1600,count_1601,count_1602,count_1603]
pct6_1600 = count_1600/(max(list))*100
pct6_1601 = count_1601/(max(list))*100
pct6_1602 = count_1602/(max(list))*100
pct6_1603 = count_1603/(max(list))*100

In [24]:
list1600 = []
list1601 = []
list1602 = []
list1603 = []
for i in range(0,len(df_16_6)):
    ans = df_16_6["answer"][i]
    ques = df_16_6["question_id"][i]
    if ques == 1600:
        list1600.append(ans)
    elif ques == 1601:
        list1601.append(ans)
    elif ques == 1602:
        list1602.append(ans)
    elif ques == 1603:
        list1603.append(ans)
    else:
        print('stray', i)
min6_00 = min(list1600)
min6_01 = min(list1601)
min6_02 = min(list1602)
min6_03 = min(list1603)

max6_00 = max(list1600)
max6_01 = max(list1601)
max6_02 = max(list1602)
max6_03 = max(list1603)

mean6_00 = s.mean(list1600)
mean6_01 = s.mean(list1601)
mean6_02 = s.mean(list1602)
mean6_03 = s.mean(list1603)

std6_00 = s.stdev(list1600)
std6_01 = s.stdev(list1601)
std6_02 = s.stdev(list1602)
std6_03 = s.stdev(list1603)

In [25]:
list1600 = []
list1601 = []
list1602 = []
list1603 = []
for i in range(0,len(df_16_27)):
    ans = df_16_27["answer"][i]
    ques = df_16_27["question_id"][i]
    if ques == 1600:
        list1600.append(ans)
    elif ques == 1601:
        list1601.append(ans)
    elif ques == 1602:
        list1602.append(ans)
    elif ques == 1603:
        list1603.append(ans)
    else:
        print('stray', i)
min27_00 = min(list1600)
min27_01 = min(list1601)
min27_02 = min(list1602)
min27_03 = min(list1603)

max27_00 = max(list1600)
max27_01 = max(list1601)
max27_02 = max(list1602)
max27_03 = max(list1603)

mean27_00 = s.mean(list1600)
mean27_01 = s.mean(list1601)
mean27_02 = s.mean(list1602)
mean27_03 = s.mean(list1603)

std27_00 = s.stdev(list1600)
std27_01 = s.stdev(list1601)
std27_02 = s.stdev(list1602)
std27_03 = s.stdev(list1603)

In [26]:
count_1600 = 0
count_1601 = 0
count_1602 = 0
count_1603 = 0

for i in range(0,len(df_16_27)):
    ques = df_16_27["question_id"][i]
    if ques == 1600:
        count_1600 +=1
    elif ques == 1601:
        count_1601 += 1
    elif ques == 1602:
        count_1602 += 1
    elif ques == 1603:
        count_1603 += 1
    else:
        print('stray', i)
list = [count_1600,count_1601,count_1602,count_1603]
pct27_1600 = count_1600/(max(list))*100
pct27_1601 = count_1601/(max(list))*100
pct27_1602 = count_1602/(max(list))*100
pct27_1603 = count_1603/(max(list))*100

In [27]:
"""pct27_1600 = count_1600/(max(list))*100
pct27_1601 = count_1601/(max(list))*100
pct27_1602 = count_1602/(max(list))*100
pct27_1603 = count_1603/(max(list))*100"""

'pct27_1600 = count_1600/(max(list))*100\npct27_1601 = count_1601/(max(list))*100\npct27_1602 = count_1602/(max(list))*100\npct27_1603 = count_1603/(max(list))*100'

In [28]:
"""min6_00 = min(list1600)
min6_01 = min(list1601)
min6_02 = min(list1602)
min6_03 = min(list1603)

 = max(list1600)
max6_01 = max(list1601)
max6_02 = max(list1602)
max6_03 = max(list1603)"""

'min6_00 = min(list1600)\nmin6_01 = min(list1601)\nmin6_02 = min(list1602)\nmin6_03 = min(list1603)\n\n = max(list1600)\nmax6_01 = max(list1601)\nmax6_02 = max(list1602)\nmax6_03 = max(list1603)'

In [30]:
base_filename = 'report.txt'
with open(os.path.join(file_dir, base_filename),'w') as outfile:
    
    outfile.write('\n\nsubject 6 Statistics \n\n')
    
    outfile.write('Fatigue = 1600 \n')
    outfile.write('Depression = 1601 \n')
    outfile.write('Anxiety = 1602 \n')
    outfile.write('Pain = 1603 \n')
    
    outfile.write('Correlation Matrix for subject 6 \n\n')
    #outfile.write(b)
    #pd.to_string(outfile)
    b.to_string(outfile)
    
    std6_00
    
    outfile.write('\n\n Fatigue')
    outfile.write('\n Maximum Value = %f'%max6_00)
    outfile.write('\n Minimum Value = %f' %min6_00)
    outfile.write('\n Mean Value = %.2f'%mean6_00)
    outfile.write('\n Standard deviation = %.2f'%std6_00)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1600)
    
    outfile.write('\n\n Depression \n')
    outfile.write('\n Maximum Value = %f'%max6_01)
    outfile.write('\n Minimum Value = %f' %min6_01)
    outfile.write('\n Mean Value = %.2f'%mean6_01)
    outfile.write('\n Standard deviation = %.2f'%std6_01)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1601)
    
    outfile.write('\n\n Anxiety \n')
    outfile.write('\n Maximum Value = %f'%max6_02)
    outfile.write('\n Minimum Value = %f' %min6_02)
    outfile.write('\n Mean Value = %.2f'%mean6_02)
    outfile.write('\n Standard deviation = %.2f'%std6_02)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1602)
    
    outfile.write('\n\n Pain \n')
    outfile.write('\n Maximum Value = %f'%max6_03)
    outfile.write('\n Minimum Value = %f' %min6_03)
    outfile.write('\n Mean Value = %.2f'%mean6_03)
    outfile.write('\n Standard deviation = %.2f'%std6_03)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1603)
    #outfile.write('Fatigue \n')
    
    outfile.write('\n\n subject 27 Statistics \n\n')
    
    outfile.write('Fatigue = 1600 \n')
    outfile.write('Depression = 1601 \n')
    outfile.write('Anxiety = 1602 \n')
    outfile.write('Pain = 1603 \n')
    
    outfile.write('Correlation Matrix for subject 27 \n\n')
    a.to_string(outfile)
    
    outfile.write('\n\n Fatigue \n')
    outfile.write('\n Maximum Value = %f'%max27_00)
    outfile.write('\n Minimum Value = %f' %min27_00)
    outfile.write('\n Mean Value = %.2f'%mean27_00)
    outfile.write('\n Standard deviation = %.2f'%std27_00)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1600)
    
    outfile.write('\n\n Depression \n')
    outfile.write('\n Maximum Value = %f'%max27_01)
    outfile.write('\n Minimum Value = %f' %min27_01)
    outfile.write('\n Mean Value = %.2f'%mean27_01)
    outfile.write('\n Standard deviation = %.2f'%std27_01)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1601)
    
    outfile.write('\n\n Anxiety \n')
    outfile.write('\n Maximum Value = %f'%max27_02)
    outfile.write('\n Minimum Value = %f' %min27_02)
    outfile.write('\n Mean Value = %.2f'%mean27_02)
    outfile.write('\n Standard deviation = %.2f'%std27_02)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1602)
    
    outfile.write('\n\n Pain \n')
    outfile.write('\n Maximum Value = %f'%max27_03)
    outfile.write('\n Minimum Value = %f' %min27_03)
    outfile.write('\n Mean Value = %.2f'%mean27_03)
    outfile.write('\n Standard deviation = %.2f'%std27_03)
    outfile.write('\n Percentage of questions answered for fatigue = %.2f' %pct6_1603)  